>[Load in Colab](#scrollTo=lm-MDLOPmwBa)

>[Initiate](#scrollTo=9OBnxzC1m1tc)

>[Prostate](#scrollTo=PQ2l2jDef9MT)

>[Create Cross folds](#scrollTo=q-fTOxiBZcm7)

>[Train](#scrollTo=dEO65i01p-bt)

>>[Train Functions](#scrollTo=zDtjJeVkBQGd)

>>[UNET](#scrollTo=nUQFtc_GoE0s)

>>[MNet](#scrollTo=vwW3vyUBoQAK)

>>[UNet3+](#scrollTo=8ADLHu-Y3l80)



# Load in Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Prostate_MRI_Detection_3D_Model"

/content/drive/.shortcut-targets-by-id/1M9aI1YrZCaL02OMqJ9ARGXDyTFCPluNH/Prostate_MRI_Detection_3D_Model


In [ ]:
!pip install -r requirement.txt
!pip install monai
!pip install dicom2nifti

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.3/91.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 85.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Initiate

In [22]:
#prostate
HOME_DIR =       "./"
DATA_DIR =       "./PROSTATEx_masks/Files/prostate/"
OUT_DIR =        "./cross_results/prostate/"
SLICED_OUT_DIR = "./data/sliced/prostate/"
#AUG_OUT_DIR = "./data/augmented_test/prostate/"
FOLD_DIR = "./cross/prostate/"

In [23]:
import os
from glob import glob
import shutil
from tqdm import tqdm
import dicom2nifti
import numpy as np
import nibabel as nib
import pandas as pd
from monai.utils import first

import json
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
import os

In [24]:
import torch
import nibabel as nib

import elasticdeform 
from skimage.util import random_noise
import scipy
from scipy.ndimage import affine_transform

import torchvision.transforms.functional as TF
import nibabel as nib
#import torchio as tio

import numpy as np
from scipy import ndimage
from scipy.ndimage import shift
import random

In [25]:
from monai.utils import first
import matplotlib.pyplot as plt
import torch
import os
import numpy as np
from monai.losses import DiceLoss
from tqdm import tqdm
from utils.train import train

In [ ]:
import torch
from monai.losses import TverskyLoss
from torch.optim import Adam

alpha = 0.5  # Tversky alpha hyperparameter
beta = 0.5  # Tversky beta hyperparameter
smooth_dr = 1e-5  # Smoothing factor to avoid division by zero

# Define a custom TverskyLoss function that handles boolean tensor
class CustomTverskyLoss(TverskyLoss):
    def forward(self, y_pred, y_true):
        # Convert boolean tensor to float before passing it to parent class
        y_true_float = y_true.float()
        return super(CustomTverskyLoss, self).forward(y_pred, y_true_float)



# Prostate 
Create 16 images 
need to run if data/sliced images aren't available

In [ ]:
def get_data_path(path):
  f = open( path + 'config.json')
  jdata = json.load(f)
  f.close()
  return jdata["path"]

In [ ]:
# create directries if does not exist
if not os.path.exists(SLICED_OUT_DIR):
    os.makedirs(SLICED_OUT_DIR)
if not os.path.exists(SLICED_OUT_DIR + "Images"):
    os.makedirs(SLICED_OUT_DIR + "Images")
if not os.path.exists(SLICED_OUT_DIR + "mask_prostate"):
    os.makedirs(SLICED_OUT_DIR + "mask_prostate")

In [ ]:
def makeLabelNo(name):
  numb= name.split("-")[1]
  li=[80,81,83,85,86,87,88,89,90,91,93,96,98,125,127]
  if(int(numb) in li):
    return 'ProstateX-' + numb[1:]
  return name

In [ ]:
df = pd.read_csv(DATA_DIR +'image_list.csv')

In [ ]:
data = [{'image': DATA_DIR + 'Images/' + x + ".nii.gz", "label":DATA_DIR + 'mask_prostate/' + makeLabelNo(x.split("_")[0]) + ".nii.gz"  } for x in df['T2']]

In [ ]:
slice_size = 16

In [ ]:
from utils.preprocessing import create_same_slice_nifti

create_same_slice_nifti(data, slice_size, SLICED_OUT_DIR)

In [ ]:
data = get_data_path(SLICED_OUT_DIR)

# Create Cross folds

need to run if 5 folds weren't created

In [18]:
def get_data_path(path):
  f = open( path )
  jdata = json.load(f)
  f.close()
  return jdata["path"]

In [19]:
def save_to_json(data, path):
  with open(path, 'w') as fp:
    json.dump(data, fp)

In [20]:
import os
if not os.path.exists(FOLD_DIR):
    os.makedirs(FOLD_DIR)

In [26]:

data = get_data_path(SLICED_OUT_DIR +  "/config.json" )
train_size =  int(len(data)*0.8)
test_paths =  data[train_size:]
save_to_json({"path": test_paths}, FOLD_DIR  + "/test_config.json")

data  = data[:train_size]

In [27]:
from sklearn.model_selection import KFold

k=5
kfold = KFold(n_splits=k)

for index, (train_indices, val_indices) in enumerate(kfold.split(data)):
  name = "fold"+str(index)
  

  if not os.path.exists(FOLD_DIR + name ):
        os.makedirs(FOLD_DIR + name)

  train_paths = []
  for index in train_indices:
    train_paths.append( data[index])

  save_to_json({"path": train_paths}, FOLD_DIR + name + "/train_config.json")

  val_paths = []
  for index in val_indices:
    val_paths.append( data[index])

  save_to_json({ "path": val_paths}, FOLD_DIR + name + "/val_config.json")
 

# Train

## Train Functions

In [16]:
from utils.transform import transform
import os.path
import torch

def updateLogs(path, data):
    f = open(path,'a')
    f.write(data)
    f.close()

def get_data_path(path):
  f = open( path )
  jdata = json.load(f)
  f.close()
  return jdata["path"]

def Cross_Val(model_class,  model_args, loss_function, load_from, fold_dir, model_dir , epochs=100, test_interval= 5,start_from= 1, device=torch.device("cpu") , start_fold=0, num_folds=5):

  for index in range(start_fold , num_folds):

    fold = 'fold'+str(index)
    
    train_data = get_data_path(fold_dir + fold +"/train_config.json")
    test_data = get_data_path(fold_dir + fold +"/val_config.json")

    pixdim =(1.5, 1.5, 1.0)
    a_min=0
    a_max=500
    spatial_size= [128, 128,16] #[384, 384,18]

    train_loader = transform(train_data, a_min, a_max, spatial_size, pixdim)
    test_loader = transform(test_data, a_min, a_max, spatial_size, pixdim)

    model_fold_dir = model_dir + fold+'/'

    if not os.path.exists(model_fold_dir):
      os.makedirs(model_fold_dir)

    data_in = [train_loader, test_loader]

    model = model_class(**model_args).to(device)
    
    path =  model_dir + fold +"/best_metric_model.pth"

    if (os.path.exists(path)):
          model.load_state_dict(torch.load(
              os.path.join(path), map_location=device))
          print("Loaded") 

    
    optimizer = torch.optim.Adam(model.parameters(), 1e-3, weight_decay=1e-5, amsgrad=True)

    
    print(f"{fold} starts_______________________\n")
    updateLogs(os.path.join(model_dir, "logs.txt"), f"{fold} starts_______________________\n")
    train(model, data_in, loss_function, optimizer, epochs, model_fold_dir, test_interval= 1,start_from= 1, load_from = model_fold_dir)
    start_from= 1
    updateLogs(os.path.join(model_dir, "logs.txt"), f"{fold} ends _______________________\n")

In [4]:
import torch 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# loss_function = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)
loss_function = CustomTverskyLoss(include_background=True, to_onehot_y=True, sigmoid=True, alpha=0.5, beta=0.5, reduction='mean')

## UNET

In [ ]:
from monai.networks.nets import UNet
from monai.networks.layers import Norm


model_dir = OUT_DIR +"unet-test_2/"
model_args = {
    'num_res_units': 2,
    'channels': (64, 128, 256, 512),
    'out_channels': 1,
    'in_channels': 1,
    'norm': Norm.BATCH,
    'spatial_dims': 3,
    'strides': (2, 2, 2),
}

Cross_Val(UNet, model_args, loss_function, fold_dir=FOLD_DIR, model_dir= model_dir, 
          epochs=2, test_interval= 1,start_from= 1, device=device, load_from = model_dir, start_fold=1, num_folds=5)

## MNet

In [ ]:
from Models.ModifiedNet import MNet

model_dir = OUT_DIR +"mnet-test/"
model_args = {
   
}

Cross_Val(MNet, model_args, loss_function, fold_dir=FOLD_DIR, model_dir= model_dir, 
          epochs=2, test_interval= 1,start_from= 1, device=device, load_from = model_dir, start_fold=1, num_folds=5)

## UNet3+

In [18]:
from Models.Unet_TriplePlus import U_net_TriplePlus

model_dir = OUT_DIR +"unet_3plus-test/"
model_args = {'in_channel':1,'out_channel':1}

Cross_Val( U_net_TriplePlus, model_args, loss_function, fold_dir=FOLD_DIR, model_dir= model_dir, 
          epochs=2, test_interval= 1,start_from= 1, device=device, load_from = model_dir, start_fold=1, num_folds=5)

c:\Users\envy\Desktop\Prostate_MRI\Prostate_MRI_Detection_3D_Model\packages\lib\site-packages\monai\utils\deprecate_utils.py:107: FutureWarning: <class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
  warn_deprecated(obj, msg, warning_category)


fold1 starts_______________________

----------
epoch 1/2


c:\Users\envy\Desktop\Prostate_MRI\Prostate_MRI_Detection_3D_Model\packages\lib\site-packages\monai\losses\dice.py:144: UserWarning: single channel prediction, `to_onehot_y=True` ignored.
  warnings.warn("single channel prediction, `to_onehot_y=True` ignored.")


1/163, Train_loss: 0.8964 Train_dice: 0.1036
2/163, Train_loss: 0.9999 Train_dice: 0.0001
3/163, Train_loss: 1.0000 Train_dice: 0.0000
4/163, Train_loss: 0.9997 Train_dice: 0.0003
5/163, Train_loss: 0.9999 Train_dice: 0.0001
6/163, Train_loss: 1.0000 Train_dice: 0.0000
7/163, Train_loss: 0.9998 Train_dice: 0.0002
8/163, Train_loss: 0.9999 Train_dice: 0.0001
9/163, Train_loss: 0.9997 Train_dice: 0.0003
10/163, Train_loss: 0.9992 Train_dice: 0.0008
11/163, Train_loss: 1.0000 Train_dice: 0.0000
12/163, Train_loss: 0.9978 Train_dice: 0.0022
13/163, Train_loss: 0.9973 Train_dice: 0.0027
14/163, Train_loss: 0.9862 Train_dice: 0.0138


# Test

# Model Loading

### MNet

In [ ]:
from Models.ModifiedNet import MNet

model_dir = OUT_DIR +"mnet-test/"
model_args = {
   
}

model = MNet(**model_args).to(device)
    
path =  model_dir + fold +"/best_metric_model.pth"

if (os.path.exists(path)):
    model.load_state_dict(torch.load(
        os.path.join(path), map_location=device))
    print("Loaded") 

### UNet

In [ ]:
from Models.UNet import UNet
model = UNet().to(device)

In [ ]:
import os.path

path = OUT_DIR+ "unet/"+ "best_metric_model.pth"

if (os.path.exists(path)):
    model.load_state_dict(torch.load(
        os.path.join(path)))
#model.eval()

### UNet 3+

In [ ]:
from Models.UNet_3Plus import UNet_3Plus

model = UNet_3Plus().to(device=device)

In [ ]:
import os.path

path = OUT_DIR+ "unet_3plus/"+ "best_metric_model.pth"

if (os.path.exists(path)):
    model.load_state_dict(torch.load(
        os.path.join(path)))
# model.eval()

## Model Testing

In [28]:
def get_data_path(path):
  f = open( path )
  jdata = json.load(f)
  f.close()
  return jdata["path"]

In [29]:
test_data = get_data_path(FOLD_DIR +"test_config.json")

In [30]:
len(test_data)

41

In [ ]:
pixdim =(1.5, 1.5, 1.0)
a_min=0
a_max=500
spatial_size= [128, 128,16] #[384, 384,18]

test_loader = transform(test_data, a_min, a_max, spatial_size, pixdim)

In [ ]:
fold = "fold0"

In [ ]:
def dice_metric(predicted, target):
    '''
    In this function we take `predicted` and `target` (label) to calculate the dice coeficient then we use it 
    to calculate a metric value for the training and the validation.
    '''
    dice_value = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)
    value = 1 - dice_value(predicted, target).item()
    return value

In [ ]:
loss_function = CustomTverskyLoss(include_background=True, to_onehot_y=True, sigmoid=True, alpha=0.5, beta=0.5, reduction='mean')

In [ ]:
model.eval()
with torch.no_grad():
  test_epoch_loss = 0
  test_metric = 0
  epoch_metric_test = 0
  test_step = 0

  for test_data in test_loader:

      test_step += 1

      test_volume = test_data["image"]
      test_label = test_data["label"]
      test_label = test_label != 0
      if torch.cuda.is_available():
          test_volume, test_label = (test_volume.to(device), test_label.to(device),)
      
      test_outputs = model(test_volume)
      
      test_loss = loss_function(test_outputs, test_label)
      test_epoch_loss += test_loss.item()
      test_metric = dice_metric(test_outputs, test_label)
      epoch_metric_test += test_metric

  epoch_metric_test /= test_step
  print(f'test_dice_epoch: {epoch_metric_test:.4f}')


In [31]:
save_to_json({"path": 'test_dice_epoch: {epoch_metric_test:.4f}'}, FOLD_DIR + name + "/results.json")